# Backtest Execution Workflow

**Systematic Macro Credit Research — Step 4 of 5**

This notebook executes backtests for selected signal-strategy pairs and saves the raw results for downstream analysis. It focuses solely on backtest execution without performance evaluation.

## Workflow Position

```
1. Data Download (01_data_download.ipynb)
   ↓
2. Signal Computation (02_signal_computation.ipynb)
   ↓
3. Signal Suitability Evaluation (03_suitability_evaluation.ipynb)
   ↓
4. Backtest Execution ← YOU ARE HERE
   ↓
5. Performance Analysis (05_performance_analysis.ipynb)
```

## Prerequisites

**This notebook only loads cached data. It does not generate any new data.**

Required files from previous steps:
- **Signals:** `data/processed/signals.parquet` (from Step 2)
- **CDX spreads:** `data/cache/bloomberg/cdx_ig_5y.parquet` (from Step 1)

If you don't have Bloomberg Terminal access, run `generate_synthetic_data.py` before Step 2 to create the required cache files.

## What This Notebook Does

1. **Load Signals** — Read computed signals from Step 2
2. **Configure Selection** — Choose signals and strategies to backtest (defaults to all)
3. **Load CDX Data** — Read spreads from cache for P&L calculation
4. **Execute Backtest Matrix** — Run backtests for all signal-strategy pairs
5. **Display Execution Summary** — Show basic execution statistics
6. **Persist Results** — Save P&L and positions for performance analysis (Step 5)

## Outputs

- **P&L Results:** `data/processed/backtest_results_pnl.parquet` (MultiIndex: signal, strategy, date)
- **Position Results:** `data/processed/backtest_results_positions.parquet` (MultiIndex: signal, strategy, date)
- **Execution Metadata:** `logs/backtest_metadata.json`

## Key Design Patterns

- **Cache-Only:** Loads all data from previous workflow steps (no data generation)
- **User Control:** Manual signal/strategy selection independent of suitability results
- **Full Matrix:** Execute all selected combinations by default
- **Execution Focus:** No performance analysis or visualization (reserved for Step 5)
- **Clean Outputs:** Raw P&L and positions ready for comprehensive analysis

---

## 1. Imports and Configuration

Import dependencies and verify data availability.

In [ ]:
import logging
from datetime import datetime
from pathlib import Path

import pandas as pd

from aponyx.config import DATA_DIR, LOGS_DIR, REGISTRY_PATH, STRATEGY_CATALOG_PATH
from aponyx.data.registry import DataRegistry
from aponyx.backtest import run_backtest
from aponyx.backtest.registry import StrategyRegistry
from aponyx.persistence import load_parquet, save_parquet, save_json

# Configure logging for notebook
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
)
logger = logging.getLogger(__name__)

print("=" * 80)
print("BACKTEST EXECUTION WORKFLOW — Step 4 of 5")
print("=" * 80)
print(f"\nConfiguration:")
print(f"  Data directory: {DATA_DIR}")
print(f"  Logs directory: {LOGS_DIR}")
print(f"  Strategy catalog: {STRATEGY_CATALOG_PATH}")
print(f"\n✓ Imports complete")

## 2. Load Signals

Load computed signals from Step 2 and display available signals.

In [ ]:
# Load signals
signals_path = DATA_DIR / "processed" / "signals.parquet"

if not signals_path.exists():
    raise FileNotFoundError(
        f"Signals file not found: {signals_path}\n"
        "Please run 02_signal_computation.ipynb first."
    )

signals = load_parquet(signals_path)

print(f"\n{'='*80}")
print(f"SIGNALS LOADED")
print(f"{'='*80}\n")
print(f"Path: {signals_path}")
print(f"Date range: {signals.index.min()} to {signals.index.max()}")
print(f"Observations: {len(signals):,}")
print(f"\nAvailable signals ({len(signals.columns)}):")
for col in signals.columns:
    print(f"  - {col}")
    
print(f"\nSignal statistics:")
print(signals.describe().to_markdown())

## 3. Configure Signal Selection

Choose which signals to backtest. Default is all available signals.

**To narrow selection:** Edit the list below to include only desired signals.

In [ ]:
# Signal selection: default to all available signals
# To narrow: selected_signals = ['cdx_vix_gap', 'spread_momentum']
selected_signals = list(signals.columns)

print(f"\n{'='*80}")
print(f"SIGNAL SELECTION")
print(f"{'='*80}\n")
print(f"Selected signals ({len(selected_signals)}):")
for signal in selected_signals:
    print(f"  ✓ {signal}")
    
# Validate selection
invalid_signals = [s for s in selected_signals if s not in signals.columns]
if invalid_signals:
    raise ValueError(f"Invalid signal names: {invalid_signals}")

## 4. Load Strategy Catalog

Load strategy definitions and display available strategies.

In [ ]:
# Load strategy registry
strategy_registry = StrategyRegistry(STRATEGY_CATALOG_PATH)
all_strategies = strategy_registry.list_all()

print(f"\n{'='*80}")
print(f"STRATEGY CATALOG")
print(f"{'='*80}\n")
print(f"Catalog path: {STRATEGY_CATALOG_PATH}")
print(f"Total strategies: {len(all_strategies)}\n")

# Display strategy catalog as table
strategy_data = []
for name, metadata in all_strategies.items():
    strategy_data.append({
        "Strategy": name,
        "Entry σ": f"{metadata.entry_threshold:.2f}",
        "Exit σ": f"{metadata.exit_threshold:.2f}",
        "Enabled": "✓" if metadata.enabled else "✗",
        "Description": metadata.description,
    })

strategy_table = pd.DataFrame(strategy_data)
print(strategy_table.to_markdown(index=False))

## 5. Configure Strategy Selection

Choose which strategies to test. Default is all strategies (including experimental).

**To narrow selection:** Edit the list below to include only desired strategies.

In [ ]:
# Strategy selection: default to all strategies
# To narrow: selected_strategies = ['conservative', 'balanced', 'aggressive']
selected_strategies = list(all_strategies.keys())

print(f"\n{'='*80}")
print(f"STRATEGY SELECTION")
print(f"{'='*80}\n")
print(f"Selected strategies ({len(selected_strategies)}):")
for strategy in selected_strategies:
    metadata = all_strategies[strategy]
    print(f"  ✓ {strategy:15s} (Entry: {metadata.entry_threshold:.2f}σ, Exit: {metadata.exit_threshold:.2f}σ)")
    
# Validate selection
invalid_strategies = [s for s in selected_strategies if s not in all_strategies]
if invalid_strategies:
    raise ValueError(f"Invalid strategy names: {invalid_strategies}")
    
print(f"\nBacktest matrix: {len(selected_signals)} signals × {len(selected_strategies)} strategies = {len(selected_signals) * len(selected_strategies)} runs")

## 6. Load CDX Spread Data

Load CDX spreads from cache for P&L calculation.

In [ ]:
# Load CDX data from cache using registry
registry = DataRegistry(REGISTRY_PATH, DATA_DIR)

print(f"\n{'='*80}")
print(f"LOADING CDX DATA FROM CACHE")
print(f"{'='*80}\n")

# Find CDX IG 5Y in registry
cdx_datasets = [
    name for name in registry.list_datasets(instrument="cdx")
    if "cdx_ig_5y" in registry.get_dataset_info(name).get("metadata", {}).get("params", {}).get("security", "")
]

if not cdx_datasets:
    raise FileNotFoundError(
        "CDX IG 5Y data not found in registry.\n"
        "Please run 01_data_download.ipynb first."
    )

# Use the first matching dataset
cdx_dataset_name = cdx_datasets[0]
cdx_info = registry.get_dataset_info(cdx_dataset_name)
cdx_cache_path = Path(cdx_info["file_path"])

print(f"Registry entry: {cdx_dataset_name}")
print(f"Cache path: {cdx_cache_path}")

cdx_df = load_parquet(cdx_cache_path)
print(f"✓ Loaded cached CDX data: {len(cdx_df):,} observations")

# Align CDX data to signal dates
cdx_spread = cdx_df['spread'].reindex(signals.index).ffill()

print(f"\nCDX spread statistics:")
print(f"  Mean: {cdx_spread.mean():.2f} bps")
print(f"  Std: {cdx_spread.std():.2f} bps")
print(f"  Min: {cdx_spread.min():.2f} bps")
print(f"  Max: {cdx_spread.max():.2f} bps")
print(f"  Missing: {cdx_spread.isna().sum()} observations")

## 7. Execute Backtest Matrix

Run backtests for all selected signal-strategy pairs.

In [ ]:
from aponyx.backtest.config import BacktestConfig

print(f"\n{'='*80}")
print(f"EXECUTING BACKTEST MATRIX")
print(f"{'='*80}\n")
print(f"Running {len(selected_signals)} signals × {len(selected_strategies)} strategies = {len(selected_signals) * len(selected_strategies)} backtests\n")

# Store backtest results
backtest_results = {}
execution_start = datetime.now()

# Execute backtest matrix
total_backtests = len(selected_signals) * len(selected_strategies)
current_backtest = 0

for signal_name in selected_signals:
    signal_series = signals[signal_name]
    
    for strategy_name in selected_strategies:
        current_backtest += 1
        print(f"[{current_backtest}/{total_backtests}] Running: {signal_name} × {strategy_name}")
        
        # Get strategy metadata
        strategy_metadata = all_strategies[strategy_name]
        
        # Create backtest configuration
        config = BacktestConfig(
            entry_threshold=strategy_metadata.entry_threshold,
            exit_threshold=strategy_metadata.exit_threshold,
            position_size=10.0,
            transaction_cost_bps=1.0,
            dv01_per_million=4750.0,
        )
        
        # Run backtest
        "        # Run backtest\n",
    "        result = run_backtest(\n",
    "            signal=signal_series,\n",
    "            spread=cdx_spread,\n",
    "            config=config,\n",
        )
        
        # Store result
        backtest_results[(signal_name, strategy_name)] = result
        
        # Display brief summary
        total_pnl = result.pnl['net_pnl'].sum()
        n_trades = (result.positions['position'].diff().abs().sum() // 2)
        print(f"  Total P&L: ${total_pnl:,.0f} | Trades: {int(n_trades)}")

execution_end = datetime.now()
execution_time = (execution_end - execution_start).total_seconds()

print(f"\n✓ Backtest matrix complete: {total_backtests} runs in {execution_time:.1f}s")
print(f"  Average: {execution_time/total_backtests:.2f}s per backtest")


## 8. Display Execution Summary

Show basic execution statistics for all backtest runs.

In [ ]:
print(f"\n{'='*80}")
print(f"EXECUTION SUMMARY")
print(f"{'='*80}\n")

# Create summary table
summary_data = []

for (signal_name, strategy_name), result in backtest_results.items():
    pnl = result.pnl['net_pnl']
    positions = result.positions
    
    summary_data.append({
        "Signal": signal_name,
        "Strategy": strategy_name,
        "Total Obs": len(pnl),
        "Total P&L": f"${pnl.sum():,.0f}",
        "# Trades": positions['position'].diff().abs().sum() // 2,
        "Days in Market": (positions['position'] != 0).sum(),
        "First Date": str(pnl.index.min().date()),
        "Last Date": str(pnl.index.max().date()),
    })

summary_df = pd.DataFrame(summary_data)

print("Backtest execution completed for all signal-strategy pairs:\n")
print(summary_df.to_markdown(index=False))

print(f"\n{'─'*80}")
print(f"Summary Statistics:")
print(f"{'─'*80}")
print(f"  Total runs: {len(backtest_results)}")
print(f"  Signals tested: {len(selected_signals)}")
print(f"  Strategies tested: {len(selected_strategies)}")
print(f"  Date range: {summary_df['First Date'].iloc[0]} to {summary_df['Last Date'].iloc[0]}")
print(f"\n✓ Backtest execution complete")
print(f"\nNext step: Run 05_performance_analysis.ipynb for comprehensive analysis")

## 9. Persist Backtest Results

Save P&L and position data for performance analysis in Step 5.

In [ ]:
print(f"\n{'='*80}")
print(f"PERSISTING BACKTEST RESULTS")
print(f"{'='*80}\n")

# Combine all results into single DataFrame with MultiIndex
all_positions = []
all_pnl = []

for (signal_name, strategy_name), result in backtest_results.items():
    # Add signal/strategy identifiers
    # Use _signal_id and _strategy_id to avoid overwriting 'signal' column
    positions_with_id = result.positions.copy()
    positions_with_id['_signal_id'] = signal_name
    positions_with_id['_strategy_id'] = strategy_name
    
    pnl_with_id = result.pnl.copy()
    pnl_with_id['_signal_id'] = signal_name
    pnl_with_id['_strategy_id'] = strategy_name
    
    all_positions.append(positions_with_id)
    all_pnl.append(pnl_with_id)

# Combine and create MultiIndex
combined_positions = pd.concat(all_positions)
combined_pnl = pd.concat(all_pnl)

# Set MultiIndex: (signal_id, strategy_id, date)
combined_positions = combined_positions.reset_index()
combined_positions = combined_positions.set_index(['_signal_id', '_strategy_id', 'date'])

combined_pnl = combined_pnl.reset_index()
combined_pnl = combined_pnl.set_index(['_signal_id', '_strategy_id', 'date'])

# Save P&L and positions separately for performance analysis
pnl_path = DATA_DIR / "processed" / "backtest_results_pnl.parquet"
positions_path = DATA_DIR / "processed" / "backtest_results_positions.parquet"

save_parquet(combined_pnl, pnl_path)
save_parquet(combined_positions, positions_path)

print(f"✓ Saved backtest P&L: {pnl_path}")
print(f"  Shape: {combined_pnl.shape}")
print(f"✓ Saved backtest positions: {positions_path}")
print(f"  Shape: {combined_positions.shape}")
print(f"  Pairs: {len(backtest_results)}")

# Save execution metadata
metadata = {
    "timestamp": datetime.now().isoformat(),
    "execution_time_seconds": execution_time,
    "configuration": {
        "selected_signals": selected_signals,
        "selected_strategies": selected_strategies,
        "position_size": 10.0,
        "transaction_cost_bps": 1.0,
        "dv01_per_million": 4750.0,
        "data_source": cdx_info.get("metadata", {}).get("cache_key", "unknown"),
    },
    "summary": {
        "total_runs": len(backtest_results),
        "date_range": {
            "start": str(signals.index.min()),
            "end": str(signals.index.max()),
        },
    },
}

metadata_path = LOGS_DIR / "backtest_metadata.json"
save_json(metadata, metadata_path)
print(f"✓ Saved execution metadata: {metadata_path}")
print(f"\n✓ All results persisted successfully")

---

## Workflow Complete

Backtest execution successful! All selected signal-strategy pairs have been tested and raw results are ready for performance analysis.

### What Was Accomplished

✓ **Signals Loaded** — Imported computed signals from Step 2  
✓ **Strategies Selected** — User-configured strategy subset (or all strategies)  
✓ **CDX Data Loaded** — Spreads loaded from cache via DataRegistry  
✓ **Backtest Matrix Executed** — All signal-strategy pairs tested  
✓ **Execution Summary Displayed** — Basic statistics for all runs  
✓ **Results Persisted** — P&L and positions saved for Step 5

### Data Flow

```
Signals (Step 2) + CDX Spreads (Step 1)
    ↓
Backtest Execution (this notebook)
├─ Strategy configuration from catalog
├─ Backtest matrix execution
└─ Raw P&L and position outputs
    ↓
Backtest Results
├─ MultiIndex: (signal, strategy, date)
├─ P&L: daily_pnl, cumulative_pnl, net_pnl
└─ Positions: signal, position, days_held, spread
    ↓
Performance Analysis (Step 5)
```

### Re-Running This Notebook

- **Signal/Strategy Selection:** Edit cells 3 and 5 to narrow focus
- **Backtest Rerun:** Results are recomputed from scratch each run
- **Outputs:** Overwrites P&L, positions, and metadata files
- **Configuration:** Modify position size, costs, or DV01 in cell 7

### Key Files Generated

```
data/
└── processed/
    ├── backtest_results_pnl.parquet (MultiIndex: signal, strategy, date)
    └── backtest_results_positions.parquet (MultiIndex: signal, strategy, date)

logs/
└── backtest_metadata.json (execution summary)
```

### Next Step

**Run `05_performance_analysis.ipynb`** to perform comprehensive post-backtest analysis:
- Extended metrics (stability, profit factor, tail ratio)
- Rolling Sharpe ratio analysis
- Return attribution (directional, signal strength, win/loss)
- Detailed visualizations (equity curves, drawdowns, attribution charts)
- Markdown report generation

### Troubleshooting

**Signals file not found:**
- Run `02_signal_computation.ipynb` first
- Verify file exists: `data/processed/signals.parquet`
- Check DATA_DIR configuration

**CDX data not found:**
- Run `01_data_download.ipynb` first
- Check registry contains CDX IG 5Y dataset
- Verify cache files exist in `data/cache/`

**Backtest execution errors:**
- Review ERROR logs for missing data or invalid configuration
- Check signal and CDX spread alignment (DatetimeIndex)
- Verify strategy catalog contains valid threshold definitions
- Ensure DV01 and position size are reasonable

**Strategy configuration issues:**
- Verify STRATEGY_CATALOG_PATH points to valid JSON file
- Check strategy thresholds are positive numbers
- Ensure selected strategies exist in catalog
- Review strategy metadata format in catalog file